In [1]:
!pip install weaviate-client langchain tiktoken pypdf rapidocr-onnxruntime

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 307.7/307.7 kB 1.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 867.6/867.6 kB 13.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 26.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 290.4/290.4 kB 20.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.9/14.9 MB 27.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 6.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 223.7/223.7 kB 20.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.3/2.3 MB 59.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 65.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 302.8/302.8 kB 34.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.7/116.7 kB 10.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 908.3/908.3 kB 11.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━

In [9]:
WEAVIATE_CLUSTER="https://rag-langchain-mistral-w7flrfit.weaviate.network"
# WEAVIATE_API_KEY="k4FFtLx0ad4pLbBm2bi11mVHdTlHVowNo99k"

In [10]:
from langchain.vectorstores import Weaviate
import weaviate

WEAVIATE_URL = WEAVIATE_CLUSTER
# WEAVIATE_API_KEY = WEAVIATE_API_KEY

client = weaviate.Client(
    url=WEAVIATE_URL, auth_client_secret=weaviate
)

/usr/local/lib/python3.10/dist-packages/weaviate/warnings.py:15: UserWarning: Auth001: The client was configured to use authentication, but weaviate is configured without
                    authentication. Are you sure this is correct?
  warnings.warn(


In [2]:
# fixing unicode error in google colab
import locale
locale.getpreferredencoding = lambda: "UTF-8"

In [3]:
!pip install sentence-transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 171.5/171.5 kB 1.8 MB/s eta 0:00:00
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl (731.7 MB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl (410.6 MB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl (121.6 MB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl (56.5 MB)
  Using cached nvidia_cusolver_cu12-11.4.5.107-py3-none-manylinux1_x86_64.whl (124.2 MB)
  Using cached nvidia_cusparse_cu12-12.1.0.106-py3-none-manylinux1_x86_64.whl (196.0 MB)
  Using cached nvidia_nccl_cu12-2.19.3-py3-none-manylinux1_x86_64.whl (166.0 MB)
  Using cached nvidia_nvtx_cu12-12.1.105-py3-none-many

In [4]:
# specify embedding model (using huggingface sentence transformer)
from langchain.embeddings import HuggingFaceEmbeddings
embedding_model_name = "sentence-transformers/all-mpnet-base-v2"
#model_kwargs = {"device": "cuda"}
embeddings = HuggingFaceEmbeddings(
  model_name=embedding_model_name,
  #model_kwargs=model_kwargs
)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.6k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/363 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/239 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

# you can load multiple types of pdf using the langchain just check with the document

https://python.langchain.com/docs/modules/data_connection/document_loaders/pdf/

In [5]:
from langchain.document_loaders import PyPDFLoader
loader = PyPDFLoader("/content/(3) AI Engineer _ Agio _ LinkedIn.pdf", extract_images=True)
pages = loader.load()

In [6]:
pages

[Document(page_content="AI Engineer\n ·  · Mid-Senior level\n201-500 employees · Information T echnology & Services\nSee recent hiring trends for Agio.  Reactivate Premium\nNo longer accepting applicationsAgio  · Toronto, ON ·  5 days ago\nRemote Full-time\n  \nAbout the job\nAgio is a hybrid managed IT and cybersecurity provider servicing the financial\nservices, health care and payments industries. The company has extensive experience\nsupporting the alternative investment space, specializing in hedge funds, private\nequity firms and asset managers. Agio offers technology hosting, monitoring,\nmanagement, helpdesk, disaster prevention and recovery, as well as managed\nsecurity, 360° cybersecurity programs, virtual CISO (vCISO) support and cybersecurity\nconsulting.\nWe have team members in the US, UK, Canada, India and the Philippines. Our remote\nwork policy allows us to accommodate our employees' need for flexibility. Our family\nforward mentality and work-life balance focus empowe

In [7]:
# Split text into chunks
from langchain.text_splitter import RecursiveCharacterTextSplitter
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=20)
docs = text_splitter.split_documents(pages)

In [8]:
docs

[Document(page_content="AI Engineer\n ·  · Mid-Senior level\n201-500 employees · Information T echnology & Services\nSee recent hiring trends for Agio.  Reactivate Premium\nNo longer accepting applicationsAgio  · Toronto, ON ·  5 days ago\nRemote Full-time\n  \nAbout the job\nAgio is a hybrid managed IT and cybersecurity provider servicing the financial\nservices, health care and payments industries. The company has extensive experience\nsupporting the alternative investment space, specializing in hedge funds, private\nequity firms and asset managers. Agio offers technology hosting, monitoring,\nmanagement, helpdesk, disaster prevention and recovery, as well as managed\nsecurity, 360° cybersecurity programs, virtual CISO (vCISO) support and cybersecurity\nconsulting.\nWe have team members in the US, UK, Canada, India and the Philippines. Our remote\nwork policy allows us to accommodate our employees' need for flexibility. Our family\nforward mentality and work-life balance focus empowe

In [11]:
vector_db = Weaviate.from_documents(
    docs, embeddings, client=client, by_text=False
)

In [16]:
print(vector_db.similarity_search("what is requirement?", k=3)[0].page_content)

cleanse, and verify the integrity of the data used for analysis
Assist in project delivery: T ake ownership of ML projects, manage timelines,
and ensure successful project delivery. Collaborate with cross-functional
teams, including data engineers, software engineers, and product managers,
to define project requirements and deliverables
Research and stay up to date: S tay abreast of the latest advancements in NLP,
machine learning, and deep learning technologies. Conduct research to
explore new methodologies, tools, and algorithms that can enhance our
existing systems
Requir ements
Bachelor's in a quantitative field
3+ years experience in a data science or machine learning engineering role
Experience programming in Python
Experience working on NLP problems using tools like spaCy and LLM
Strong problem-solving skills: Ability to identify business problems, translate
them into data-driven solutions, and deliver actionable insights. Excellent
analytical and critical thinking skills.


In [17]:
print(vector_db.similarity_search("what is requirement?", k=3)[1].page_content)

Disability insurance
Qualifications4/22/24, 9:35 AM (3) AI Engineer | Agio | LinkedIn
https://www.linkedin.com/jobs/view/ai-engineer-at-agio-3901597819/?originalSubdomain=ca 2/5


In [18]:
print(vector_db.similarity_search("what is requirement?", k=3)[2].page_content)

See mor e jobs lik e this
About Accessibility Talent Solutions
Community Guidelines Careers Mark eting Solutions
Ad Choices Advertising
Sales Solutions Mobile Small Business
Safety Cent erQuestions?
Visit our Help Center.
Manage y our account and priv acy
Go to your Settings.
Recommendation transp arency
Learn more about R ecommended Content.Select Language
LinkedIn Corporation © 2024
22 comp any alumni w ork her e
6 days ago
4 school alumni w ork her e
1 week ago Easy Apply
 LEARNING
Learn skills to get a new job with these courses
336,540 viewers
279,754 viewers
132,916 viewers
Show mor e on Link edIn Learning
Professional Netw orking
Developing Y our Pr ofessional Image in a New Job
The New Rules o f Work
Looking for talent? Post a job
Privacy & T erms  
English (English)4/22/24, 9:35 AM (3) AI Engineer | Agio | LinkedIn
https://www.linkedin.com/jobs/view/ai-engineer-at-agio-3901597819/?originalSubdomain=ca 5/5UBC


In [19]:
print(
    vector_db.similarity_search(
        "what is requirement?", k=3)
    )

[Document(page_content="cleanse, and verify the integrity of the data used for analysis\nAssist in project delivery: T ake ownership of ML projects, manage timelines,\nand ensure successful project delivery. Collaborate with cross-functional\nteams, including data engineers, software engineers, and product managers,\nto define project requirements and deliverables\nResearch and stay up to date: S tay abreast of the latest advancements in NLP,\nmachine learning, and deep learning technologies. Conduct research to\nexplore new methodologies, tools, and algorithms that can enhance our\nexisting systems\nRequir ements\nBachelor's in a quantitative field\n3+ years experience in a data science or machine learning engineering role\nExperience programming in Python\nExperience working on NLP problems using tools like spaCy and LLM\nStrong problem-solving skills: Ability to identify business problems, translate\nthem into data-driven solutions, and deliver actionable insights. Excellent\nanalyt

In [20]:
from langchain.prompts import ChatPromptTemplate

template="""You are an assistant for question-answering tasks.
Use the following pieces of retrieved context to answer the question.
If you don't know the answer, just say that you don't know.
Use ten sentences maximum and keep the answer concise.
Question: {question}
Context: {context}
Answer:
"""

In [21]:
prompt=ChatPromptTemplate.from_template(template)

In [22]:
prompt

ChatPromptTemplate(input_variables=['context', 'question'], messages=[HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['context', 'question'], template="You are an assistant for question-answering tasks.\nUse the following pieces of retrieved context to answer the question.\nIf you don't know the answer, just say that you don't know.\nUse ten sentences maximum and keep the answer concise.\nQuestion: {question}\nContext: {context}\nAnswer:\n"))])

In [23]:
from langchain import HuggingFaceHub

In [25]:
from google.colab import userdata
huggingfacehub_api_token=userdata.get('HuGGINGFACE_TOKEN')

In [27]:
model = HuggingFaceHub(
    huggingfacehub_api_token=huggingfacehub_api_token,
    repo_id="mistralai/Mistral-7B-Instruct-v0.1",
    model_kwargs={"temperature":1, "max_length":180}
)

In [28]:
from langchain.schema.runnable import RunnablePassthrough
from langchain.schema.output_parser import StrOutputParser

In [29]:
output_parser=StrOutputParser()

In [30]:
retriever=vector_db.as_retriever()

In [31]:
rag_chain = (
    {"context": retriever,  "question": RunnablePassthrough()}
    | prompt
    | model
    | output_parser
)

In [32]:
print(rag_chain.invoke("what is job requirement?"))

Human: You are an assistant for question-answering tasks.
Use the following pieces of retrieved context to answer the question.
If you don't know the answer, just say that you don't know.
Use ten sentences maximum and keep the answer concise.
Question: what is job requirement?
Context: [Document(page_content="cleanse, and verify the integrity of the data used for analysis\nAssist in project delivery: T ake ownership of ML projects, manage timelines,\nand ensure successful project delivery. Collaborate with cross-functional\nteams, including data engineers, software engineers, and product managers,\nto define project requirements and deliverables\nResearch and stay up to date: S tay abreast of the latest advancements in NLP,\nmachine learning, and deep learning technologies. Conduct research to\nexplore new methodologies, tools, and algorithms that can enhance our\nexisting systems\nRequir ements\nBachelor's in a quantitative field\n3+ years experience in a data science or machine learn